In [ ]:
%pip install langchain
%pip install openai
%pip install PyPDF2
%pip install faiss-cpu

In [2]:
import os
import openai
import pinecone
from rate_limiter import Api
from uuid import uuid4
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

ImportError: cannot import name 'API' from 'rate_limiter' (d:\Documents\programming\repositories\CIXER\rate_limiter.py)

In [35]:
# Get your API keys from openai, you will need to create an account. 
load_dotenv()
key_openai = os.getenv("key_openai")
key_pinecone = os.getenv("key_pinecone")
idx_pinecone = os.getenv("env_pinecone")

In [40]:
# location of the pdf file/files.
locations = [
  'C:/Users/shaj6/Downloads/lululemon-2022-annual-report.pdf',
  'C:/Users/shaj6/Downloads/2021-lululemon-impact-report-final-13-09-22.pdf',
  'C:/Users/shaj6/Downloads/lululemon-athletica-inc-ar2021.pdf',
  'C:/Users/shaj6/Downloads/code-of-conduct-november-2021-english.pdf'   
]
readers = []
for loc in locations:
    readers.append(PdfReader(loc))

In [41]:
readers

In [43]:
# read data from the file and put them into a variable called raw_text
files = []
for idx, reader in enumerate(readers):
    raw_text = ''
    for i, page in enumerate(reader.pages):
        text = page.extract_text()
        if text:
            raw_text += text
    obj = {'raw_text':raw_text, 'file_location':locations[idx]}
    files.append(obj)

In [28]:
print(len(files[0]['raw_text']))

316460


In [44]:
# Chunk input text
text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
for file in files:
    raw_text = file['raw_text']
    file['chunks'] = text_splitter.split_text(raw_text)

In [48]:
count = 0
for file in files:
  count += len(file['chunks'])
print(count)

1071


In [47]:
# Convert to embeddings using OpenAI
vdb = pinecone.Index(idx_pinecone)
api = API(average_rate_limit=10, max_retries=5)
for file in files:
   doc_id = uuid4()
   payload = []
   for i, chunk in enumerate(file['chunks']):
      # create vector id
      vec_id = uuid4()

      # get vector representation of text chunk
      chunk = chunk.encode(encoding='ASCII',errors='ignore').decode()  # fix any UNICODE errors
      response = openai.Embedding.create(input=chunk,engine='text-embedding-ada-002')
      vector_value = response['data'][0]['embedding']  # this is a normal list

      # vector metadata as dictionary
      metadata = {
         'document_id':doc_id,
         'file_location': file['file_location'],
         'chunk_index': i
      }

      # create and append vector obj to the payload
      vector_obj = (vec_id, vector_value, metadata)
      payload.append(vector_obj)

      # status update
      if (i % 10 == 0):
         print(f"File {doc_id}, chunk {i}")
   
   vdb.upsert(payload)

File e344a624-ee5b-49a6-a4fd-7d2494ec33e1, chunk 0


APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
docsearch = FAISS.from_texts(texts, embeddings)

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [ ]:
query = "who are the authors of the article?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The authors of the article are Yuvanesh Anand, Zach Nussbaum, Brandon Duderstadt, Benjamin Schmidt and Andriy Mulyar.'

In [ ]:
query = "What was the cost of training the GPT4all model?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' $100'

In [ ]:
query = "How was the model trained?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The model was trained with LoRA (Hu et al., 2021) on the 437,605 post-processed examples for four epochs. Detailed model hyper-parameters and training code can be found in the associated repository and model training log.'

In [ ]:
query = "what was the size of the training dataset?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The final training dataset contains 437,605 prompt-generation pairs.'

In [ ]:
query = "How is this different from other models?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' This model is different from other models because it is based on LLaMA, it is licensed only for research purposes, and it is trained on a dataset of post-processed examples. It also has a TSNE visualization of the final training data, and a zoomed-in view to show generations related to personal health and wellness.'

In [ ]:
query = "What is Google Bard?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" I don't know."